In [18]:
import pandas as pd
import os
import numpy as np

In [17]:
folder = "./data"

files = os.listdir(folder)

datasets = {file_name.replace(".xlsx", ""): pd.read_excel(os.path.join(folder, file_name)) for file_name in files}

for name, df in datasets.items():
    print(f"\n{name} loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.head())


EOAL_PAIS_DL loaded: 943 rows, 7 columns
   Unnamed: 0   AÑO  MES       PAIS             LUGAR_RESIDENCIA  VIAJEROS  \
0           0  2022    1   Alemania  Residentes en el Extranjero      2892   
1           1  2022    1    Austria  Residentes en el Extranjero         0   
2           2  2022    1    Bélgica  Residentes en el Extranjero       717   
3           3  2022    1  Dinamarca  Residentes en el Extranjero         0   
4           4  2022    1     España         Residentes en España     39580   

   PERNOCTACIONES  
0            6133  
1               0  
2            1603  
3               0  
4           86870  

CST_GASTO_CONSUMO_TURISTICO_DL loaded: 112 rows, 9 columns
   Unnamed: 0   AÑO                                         PRODUCTO  \
0           0  2016  Agencias de viajes y otros servicios de reserva   
1           1  2016                              Hoteles y similares   
2           2  2016               Otros productos no característicos   
3           3  2016  

In [8]:
datasets["FRONTUR_DL"]["TIPO_VISITANTE"].unique().tolist()

['Excursionista', 'Turista']

In [9]:
datasets["EGATUR_DL"].apply(lambda col: col.unique())

AÑO                 [2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
MES                          [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
PAIS_RESIDENCIA    [Alemania, Bélgica, EEUU, Francia, Irlanda, It...
TIPO_VISITANTE     [Excursionista no residente (no tránsito), Exc...
CCAA_DESTINO       [Andalucía, Canarias, Cataluña, Illes Balears,...
PERNOCTACIONES     [0, 876, 59, 1826, 36, 220, 22, 645, 13, 576, ...
GASTO_TOTAL        [306434.84, 14063908.66, 549930.72, 956234.72,...
dtype: object

In [10]:
datasets["EGATUR_DL"]["PAIS_RESIDENCIA"].unique().tolist()

['Alemania',
 'Bélgica',
 'EEUU',
 'Francia',
 'Irlanda',
 'Italia',
 'Países Bajos',
 'Países Nórdicos (Dinamarca',
 'Portugal',
 'Reino Unido',
 'Resto de América',
 'Resto de Europa',
 'Resto del mundo',
 'Rusia',
 'Suiza']

In [15]:
datasets["CST_GASTO_CONSUMO_TURISTICO_DL"].columns.tolist()

['Unnamed: 0',
 'AÑO',
 'PRODUCTO',
 'NUM_GASTO_INTERIOR',
 'NUM_GASTO_INTERNO',
 'NUM_GASTO_EMISOR',
 'NUM_CONSUMO_TURISTICO_INTERIOR',
 'NUM_CONSUMO_OTROS_COMPONENTES_TURISTICOS',
 'NUM_GASTO_RECEPTOR']

In [16]:
datasets["CST_GASTO_CONSUMO_TURISTICO_DL"].head()

,Unnamed: 0,AÑO,PRODUCTO,NUM_GASTO_INTERIOR,NUM_GASTO_INTERNO,NUM_GASTO_EMISOR,NUM_CONSUMO_TURISTICO_INTERIOR,NUM_CONSUMO_OTROS_COMPONENTES_TURISTICOS,NUM_GASTO_RECEPTOR
0,0,2016,Agencias de viajes y otros servicios de reserva,6910.6,5090.6,3509.2,7021.3,110.7,1819.9
1,1,2016,Hoteles y similares,23393.6,9216.4,4030.7,23467.8,74.2,14177.2
2,2,2016,Otros productos no característicos,28999.0,8033.0,3929.3,28999.0,0.0,20966.0
3,3,2016,Productos característicos del turismo,86710.1,39006.0,15699.0,102669.9,15959.8,47704.1
4,4,2016,Servicios culturales,5383.0,812.3,202.7,5663.4,280.4,4570.7


In [13]:
datasets["EOAC_CATEGORIA_DL"]["CATEGORIA"].unique().tolist()

['2ª categoría', '3ª categoría', 'Lujo y 1ª categoría']